In [ ]:
%pip install --upgrade --quiet  networkx

In [2]:
%pip install langchain-together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 11.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.5/389.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1


In [3]:
pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_community.graphs.index_creator import GraphIndexCreator

from langchain_together import ChatTogether

In [6]:
chat_model = ChatTogether(

    together_api_key="b441821a89000939d800502a2aa4a7b72c8749ae33a48cfe2f42625c558f600a",

    model="meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",

    temperature = 0

)

index_creator = GraphIndexCreator(llm=chat_model)

In [7]:
!pip install pandas python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 6.0 MB/s eta 0:00:00ta 0:00:01


In [8]:
# Step 1: List files in the dataset directory
import os

dataset_dir = '/kaggle/input/fd-dataset1/'
files = os.listdir(dataset_dir)
for file in files:
    print(file)

# Step 2: Read the Word file if it exists
from docx import Document

# Path to the Word file in the Kaggle dataset
word_file_path = '/kaggle/input/fd-dataset1/FD.docx'

# Read the Word file
doc = Document(word_file_path)

# Extract text from the document
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)

# Join all paragraphs into a single string
document_text = '\n'.join(full_text)
#print(document_text)

FD.docx


In [9]:
all_text = str(document_text)

In [10]:
all_text=all_text[:4000]

In [11]:
len(all_text)

4000

In [12]:
graph = index_creator.from_text(all_text)

In [13]:
graph.get_triples()

[]

In [14]:
from langchain.chains import GraphQAChain
from langchain.prompts import PromptTemplate

In [15]:
qa_generation_template = """
You are an AI Agent specialized in retrieving information about Bajaj Finance's fixed deposit options.
The user has provided a question, and your task is to suggest the most relevant answer based on the graph context.
Interest rates, schemes, and descriptions are provided in the context as [c: interest rate, d: description, ...].

Instructions:
- Analyze the user's question.
- Return the most relevant details and their descriptions from the context provided.
- Provide your reasoning for why each detail is relevant to the question.
- Do not return the prompt, the question, or the context, just the answer.

Question: {question}
Context: {context}

Answer:
"""

entity_prompt_template = """
You are tasked with extracting relevant information about Bajaj Finance's fixed deposit options from a graph based on the user's query.
Each node represents an interest rate, a fixed deposit scheme, or a customer type. Your goal is to list out all relevant details, restricting your answer **only** to what exists in the graph.
If the query contains terms that closely match words in the context, match those words to the context.

Here are the extracted triples from the graph:
{context}

Please provide the extracted details along with relationships between them.

Extracted Entities:
"""


# Define the prompt template instances
qa_generation_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=qa_generation_template
)

entity_prompt = PromptTemplate(
    input_variables=["context"],
    template=entity_prompt_template
)

# Initialize the GraphQAChain with the LLM, graph, and refined prompt templates
chain = GraphQAChain.from_llm(
    llm=chat_model,
    graph=graph,
    qa_prompt=qa_generation_prompt,
    entity_prompt=entity_prompt,
    verbose=True
)

In [16]:
chain.run("What is SDP?")

/tmp/ipykernel_33/2900122201.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("What is SDP?")




> Entering new GraphQAChain chain...
Entities Extracted:
Based on the query "What is SDP?", I will extract the relevant information from the graph. Here are the extracted details along with relationships between them:

**SDP (Short Duration Product)**

* **Type**: Fixed Deposit Scheme
* **Interest Rate**: 7.50% (for a specific tenure, e.g., 1 year)
* **Customer Type**: Retail Customers
* **Tenure**: 1 year
* **Minimum Deposit**: ₹ 25,000
* **Maximum Deposit**: ₹ 1 Crore
* **Frequency of Interest**: Quarterly
* **Tax Benefits**: Eligible for tax benefits under Section 80C of the Income Tax Act, 1961

Please note that the details provided are based on the assumption that the graph contains information about Bajaj Finance's SDP product. If the graph contains different information, the extracted details may vary accordingly.
Full Context:


> Finished chain.


'Based on the context provided, I found the following relevant details:\n\n* [c: 7.50%, d: Senior Citizen Deposit Plan (SDP) offers a higher interest rate of 7.50% for senior citizens.]\n\nMy reasoning is that the question asks about SDP, and the context provides a specific interest rate and description for the Senior Citizen Deposit Plan, which matches the query.'

In [17]:
chain.run("For what tenure is the max interest rate in Bajaj FD?")



> Entering new GraphQAChain chain...
Entities Extracted:
Based on the query "For what tenure is the max interest rate in Bajaj FD?", I will extract the relevant details from the graph.

**Extracted Entities:**

1. **Entity Type:** Interest Rate
2. **Entity Name:** Max Interest Rate
3. **Relationship:** Associated with
4. **Entity Type:** Tenure
5. **Entity Name:** (Unknown, to be determined)

**Extracted Relationships:**

1. Max Interest Rate is associated with Bajaj FD.
2. Bajaj FD is associated with Tenure.

**Extracted Details:**

1. The max interest rate is associated with Bajaj FD.
2. Bajaj FD has a relationship with Tenure.

**Determined Entity:**

After analyzing the graph, I found that the max interest rate in Bajaj FD is associated with a tenure of **5 years**.

**Final Extracted Details:**

1. Entity Type: Interest Rate
	* Entity Name: Max Interest Rate
	* Relationship: Associated with
	* Entity Type: Fixed Deposit Scheme
	* Entity Name: Bajaj FD
2. Entity Type: Fixed Depos

"Based on the context, the maximum interest rate for Bajaj Finance's fixed deposit is offered for a tenure of 5 years.\n\nReasoning:\n- The context provides interest rates for different tenures, and the highest interest rate is mentioned for the 5-year tenure.\n- This suggests that the 5-year tenure is the optimal choice for maximizing returns on investment in Bajaj Finance's fixed deposit."

In [18]:
chain.run("What are the charges for premature FD withdrawal?")



> Entering new GraphQAChain chain...
Entities Extracted:
Based on the query "What are the charges for premature FD withdrawal?", I have extracted the following details from the graph:

**Extracted Entities:**

1. **Premature FD Withdrawal**
	* Relationship: Has charge
2. **Charge**
	* Relationship: Associated with Premature FD Withdrawal
3. **Bajaj Finance**
	* Relationship: Offers Premature FD Withdrawal (implied, not explicitly stated in the query)

**Extracted Details:**

* The charge for premature FD withdrawal is not explicitly stated in the query, but it is implied to be associated with the premature FD withdrawal.
* Bajaj Finance is the entity that offers the premature FD withdrawal option.

Please note that the graph does not provide explicit information about the charge for premature FD withdrawal, so I have only extracted the implied relationships and entities.
Full Context:


> Finished chain.


"There are no charges for premature FD withdrawal if the deposit is withdrawn after 5 years from the date of deposit. However, if the deposit is withdrawn before 5 years, a penalty of 0.50% to 1% of the deposit amount will be charged.\n\nThe details provided are relevant to the question because they directly address the charges associated with premature FD withdrawal, which is the main concern of the user's query."

In [19]:
chain.run("What are senior citizen FD rates at Bajaj Finance and for what tenure?")



> Entering new GraphQAChain chain...
Entities Extracted:
Based on the query, I will extract the relevant information from the graph. Here are the extracted details along with relationships between them:

**Extracted Entities:**

1. **Senior Citizen FD**
	* Relationship: Type of Fixed Deposit Scheme
2. **Bajaj Finance**
	* Relationship: Issuer of Fixed Deposit
3. **Interest Rate**
	* Relationship: Associated with Senior Citizen FD
4. **Tenure**
	* Relationship: Associated with Senior Citizen FD

**Extracted Details:**

* Senior Citizen FD is a type of Fixed Deposit Scheme offered by Bajaj Finance.
* The interest rate for Senior Citizen FD is not explicitly mentioned in the query, but it is implied to be relevant to the answer.
* The tenure for Senior Citizen FD is also not explicitly mentioned in the query, but it is implied to be relevant to the answer.

**Possible Answer:**

Unfortunately, the query does not provide enough information to extract specific interest rates and tenures f

'Senior Citizen FD rates at Bajaj Finance are as follows:\n- 7.50% for 36 months\n- 7.75% for 60 months\n- 8.00% for 84 months\n\nThese rates are relevant to the question because they specifically mention senior citizen FD rates, which is what the user is inquiring about. The tenures mentioned (36 months, 60 months, and 84 months) are also relevant as they provide the user with information on the available options for different investment periods.'

In [20]:
chain.run("42 months ki FD ka interest rate kya hai senior citizen ka bajaj finance me?")



> Entering new GraphQAChain chain...
Entities Extracted:
Based on the query, I will extract the relevant information from the graph. Here are the extracted details along with relationships between them:

**Extracted Entities:**

1. **Interest Rate**: 42 months ki FD ka interest rate
	* Relationship: Associated with
	* Value: (Not explicitly mentioned in the query, but we can infer it from the graph)
2. **Fixed Deposit Scheme**: 42 months ki FD
	* Relationship: Associated with
	* Value: (Not explicitly mentioned in the query, but we can infer it from the graph)
3. **Customer Type**: Senior Citizen
	* Relationship: Associated with
	* Value: (Not explicitly mentioned in the query, but we can infer it from the graph)
4. **Financial Institution**: Bajaj Finance
	* Relationship: Associated with
	* Value: (Not explicitly mentioned in the query, but we can infer it from the graph)

**Extracted Information:**

Based on the graph, I can extract the following information:

* The interest rate f

'[c: 6.50%, d: Senior Citizen FD Scheme for 42 months]\n[c: 6.35%, d: Senior Citizen FD Scheme for 36 months]\n[c: 6.30%, d: Senior Citizen FD Scheme for 30 months]\n\nReasoning:\nThe question asks for the interest rate of a 42-month fixed deposit for senior citizens at Bajaj Finance. The context provides interest rates for various fixed deposit schemes, including those specifically for senior citizens. The most relevant details are the interest rates for the 42-month, 36-month, and 30-month senior citizen FD schemes, as they directly address the question.'

In [22]:
chain.run("42 months FD interest rate for senior citizen at Bajaj Finance please?")



> Entering new GraphQAChain chain...
Entities Extracted:
Based on the query "42 months FD interest rate for senior citizen at Bajaj Finance please?", I will extract the relevant information from the graph. Here are the extracted details along with relationships between them:

**Extracted Entities:**

1. **Interest Rate**: 42 months FD interest rate
	* Relationship: Associated with
	* Entity: Bajaj Finance
2. **Fixed Deposit Scheme**: 42 months FD
	* Relationship: Has interest rate
	* Entity: Bajaj Finance
3. **Customer Type**: Senior Citizen
	* Relationship: Eligible for
	* Entity: 42 months FD
4. **Financial Institution**: Bajaj Finance
	* Relationship: Offers
	* Entity: 42 months FD

**Extracted Details:**

* Interest Rate: 42 months FD interest rate
* Fixed Deposit Scheme: 42 months FD
* Customer Type: Senior Citizen
* Financial Institution: Bajaj Finance

Note that the extracted details only include information that exists in the graph and are relevant to the query.
Full Context:

'Based on the context provided, I found the following relevant details:\n\n* 5 years: [c: 7.50%, d: For Senior Citizens]\n* 3 years: [c: 7.25%, d: For Senior Citizens]\n* 5 years: [c: 7.50%, d: For General Public]\n* 3 years: [c: 7.25%, d: For General Public]\n\nMy reasoning is that the question asks for the interest rate for a 42-month FD specifically for senior citizens at Bajaj Finance. However, the exact duration of 42 months is not mentioned in the context. The closest durations mentioned are 3 years and 5 years. Since the question is specifically for senior citizens, I have selected the interest rates for 3 years and 5 years for senior citizens from the context.'